In [10]:
import pandas as pd           # Used for data handling
import os                     # Used for checking file existence
from typing import Dict, Any  # Used for function typing hints
from IPython.display import display # Used for displaying dataframes

# Define the data structure for a single applicant (used for typing hints)
Applicant = Dict[str, Any]

def analyze_risk(applicant: Applicant) -> str:
    """
    Analyzes an applicant's profile using a set of predefined, hard-coded rules
    to categorize their credit risk as 'Low Risk', 'Medium Risk', or 'High Risk'.

    *** YOUR TASK: TUNE THESE THRESHOLDS! ***
    These initial values are educated guesses. To maximize your project's accuracy,
    you must change the numbers and purpose lists based on your 'Training Data' analysis.
    """

    # --- Extract Key Metrics from the DataFrame Row ---
    # NOTE: These names match the column headers in your CSV file
    credit_amount = applicant.get('Credit amount', 999999) 
    duration = applicant.get('Duration', 60) 
    loan_purpose = str(applicant.get('Purpose', '')).lower() 
    
    # =======================================================
    # RULE SET 1: HIGH RISK (Designed to predict 'bad' outcomes)
    # =======================================================

    # Rule H1: High Credit Amount
    # TUNE THIS: If the requested credit is very high, the financial strain and risk are greater.
    HIGH_AMOUNT_THRESHOLD = 5000 
    if credit_amount > HIGH_AMOUNT_THRESHOLD:
        return "High Risk (H1)"

    # Rule H2: Long Duration
    # TUNE THIS: Longer repayment times introduce more uncertainty and chance of life changes.
    LONG_DURATION_THRESHOLD = 48 
    if duration > LONG_DURATION_THRESHOLD:
        return "High Risk (H2)"

    # Rule H3: High-Risk Purpose 
    # Loans for speculative or non-essential purposes are often deemed high-risk.
    HIGH_RISK_PURPOSES = ['business', 'other', 'repairs', 'vacation']
    if loan_purpose in HIGH_RISK_PURPOSES:
        return "High Risk (H3)"

    # =======================================================
    # RULE SET 2: LOW RISK (Designed to predict 'good' outcomes)
    # =======================================================

    # Rule L1: Low Credit Amount AND Short Duration
    # Small loans paid back quickly are generally the safest.
    LOW_AMOUNT_THRESHOLD = 1500
    SHORT_DURATION_THRESHOLD = 18 
    if credit_amount < LOW_AMOUNT_THRESHOLD and duration <= SHORT_DURATION_THRESHOLD:
        return "Low Risk (L1)"

    # Rule L2: Safe Purpose
    # TUNE THIS: Some purposes (like 'radio/TV') might be very safe.
    SAFE_PURPOSES = ['radio/tv', 'domestic appliances', 'education', 'car']
    if loan_purpose in SAFE_PURPOSES:
        return "Low Risk (L2)"

    # =======================================================
    # RULE SET 3: MEDIUM RISK (Catch-all)
    # =======================================================

    # If the applicant didn't trigger any explicit High or Low risk rules, they fall here.
    return "Medium Risk"

In [11]:
try:
    if os.path.exists(FILE_PATH):
        df_applicants = pd.read_csv(FILE_PATH)
        print(f"Successfully loaded data from: {FILE_PATH}")
    else:
        raise FileNotFoundError(f"File not found at: {FILE_PATH}")

except FileNotFoundError:
    print(f"FATAL ERROR: Could not find or load file at {FILE_PATH}. Check the PATH_ELEMENTS list.")
    df_applicants = pd.DataFrame()
except Exception as e:
    print(f"ERROR: Problem loading CSV contents: {e}.")
    df_applicants = pd.DataFrame()

# --- 3. Column Safety Override ---
if not df_applicants.empty:
    print(f"Total applicants loaded: {len(df_applicants)}")
    
    # Check for the required 'Risk' column (or 'Class' as a common alternative)
    is_risk_column_present = RISK_COLUMN_NAME in df_applicants.columns or 'Class' in df_applicants.columns

    if not is_risk_column_present:
        # **SAFETY OVERRIDE**: If the outcome column is missing, create a dummy column.
        # This allows Cell 4 to run, but the accuracy will be meaningless.
        df_applicants[RISK_COLUMN_NAME] = 'Good' 
        print(f"\n! WARNING: Outcome column '{RISK_COLUMN_NAME}' was MISSING. Added placeholder column.")
        print("! ACCURACY MEASUREMENTS WILL NOT BE VALID until you use a file containing the true outcome.")
    else:
        # Rename 'Class' column to 'Risk' if it exists
        if 'Class' in df_applicants.columns:
            df_applicants.rename(columns={'Class': RISK_COLUMN_NAME}, inplace=True)
            
    print("Initial Applicant Data (Head, showing key features and the actual outcome):")
    # This will now display the 'Risk' column, whether it's real or the placeholder.
    display(df_applicants[['Credit amount', 'Duration', 'Purpose', RISK_COLUMN_NAME]].head())
    print("\n" + "="*50 + "\n")

Successfully loaded data from: C:\Users\Joel_\Downloads\AI-assessment-1\test_data.csv
Total applicants loaded: 280

! WARNING: Outcome column 'Risk' was MISSING. Added placeholder column.
! ACCURACY MEASUREMENTS WILL NOT BE VALID until you use a file containing the true outcome.
Initial Applicant Data (Head, showing key features and the actual outcome):


,Credit amount,Duration,Purpose,Risk
0,1893,12,car,Good
1,1829,15,radio/TV,Good
2,5848,36,radio/TV,Good
3,6527,60,car,Good
4,2910,24,car,Good


In [12]:
# --- CELL 3: APPLY THE RULE-BASED ALGORITHM ---

if not df_applicants.empty:
    # Use the .apply() method to execute the analyze_risk function for every row.
    # The result is stored in a new column called 'Predicted_Risk'.
    df_applicants['Predicted_Risk'] = df_applicants.apply(analyze_risk, axis=1)
    
    print("Classification Complete. Here is a sample of the results:")
    # We display the predicted risk next to the actual risk (the 'Risk' column).
    display(df_applicants[['Credit amount', 'Duration', 'Purpose', RISK_COLUMN_NAME, 'Predicted_Risk']].head(10))
    print("\n" + "="*50 + "\n")
else:
    print("Cannot apply algorithm. Dataframe is empty.")

Classification Complete. Here is a sample of the results:


,Credit amount,Duration,Purpose,Risk,Predicted_Risk
0,1893,12,car,Good,Low Risk (L2)
1,1829,15,radio/TV,Good,Low Risk (L2)
2,5848,36,radio/TV,Good,High Risk (H1)
3,6527,60,car,Good,High Risk (H1)
4,2910,24,car,Good,Low Risk (L2)
5,1217,18,domestic appliances,Good,Low Risk (L1)
6,10722,47,car,Good,High Risk (H1)
7,3914,48,business,Good,High Risk (H3)
8,750,18,education,Good,Low Risk (L1)
9,1347,10,radio/TV,Good,Low Risk (L1)


In [13]:
if not df_applicants.empty and RISK_COLUMN_NAME in df_applicants.columns:
    
    # We must map the actual outcomes (e.g., 'Good'/'Bad') to the categories the algorithm produces (Low/High)
    
    # Convert actual outcomes to simplified categories
    def map_actual_risk(actual_outcome):
        """Maps the dataset's 'Good' or 'Bad' label to the algorithm's 'Low' or 'High' categories."""
        # Note: If your true outcome is 1/0 instead of 'Good'/'Bad', this function needs adjustment!
        if actual_outcome in ['Good', 1]:
            return 'Low Risk'
        elif actual_outcome in ['Bad', 0]:
            return 'High Risk'
        return 'Medium Risk' 

    # Simplify the predicted risk for comparison (strips codes like '(H1)' from the prediction)
    df_applicants['Simple_Prediction'] = df_applicants['Predicted_Risk'].str.split(' ').str[0] + ' Risk'
    df_applicants['Simple_Actual'] = df_applicants[RISK_COLUMN_NAME].apply(map_actual_risk)

    # 1. Calculate overall accuracy
    correct_predictions = (df_applicants['Simple_Prediction'] == df_applicants['Simple_Actual']).sum()
    total_predictions = len(df_applicants)
    overall_accuracy = (correct_predictions / total_predictions) * 100

    print(f"--- RULE-BASED ALGORITHM PERFORMANCE ---")
    print(f"Overall Accuracy on Test Data: {overall_accuracy:.2f}%")
    print("-" * 40)
    
    # 2. Show a confusion matrix for detailed performance
    print("Classification Matrix (Predicted vs. Actual):")
    # This matrix shows how many correct and incorrect predictions were made for Low vs. High Risk
    display(pd.crosstab(df_applicants['Simple_Actual'], df_applicants['Simple_Prediction'], margins=True))

else:
    print("Cannot calculate accuracy. Data not loaded or the true outcome column ('Risk' or placeholder) is missing.")

--- RULE-BASED ALGORITHM PERFORMANCE ---
Overall Accuracy on Test Data: 55.00%
----------------------------------------
Classification Matrix (Predicted vs. Actual):


Simple_Prediction,High Risk,Low Risk,Medium Risk,All
Simple_Actual,,,,
Low Risk,85,154,41,280
All,85,154,41,280


In [16]:
# --- NEW CELL: CORRECTED TRAINING DATA ANALYSIS ---

# **CRITICAL**: REPLACE THIS PLACEHOLDER PATH with the absolute path to your training_data.csv file.
FILE_PATH_TRAINING = r'C:\Users\Joel_\Downloads\AI-assessment-1\training_data.csv' 
RISK_COLUMN_NAME = 'Risk' 

try:
    # Use the robust absolute path provided by the user (assuming they replace the placeholder)
    df_train = pd.read_csv(FILE_PATH_TRAINING)
    
    # --- Data Preparation for Analysis ---
    if 'Class' in df_train.columns:
        df_train.rename(columns={'Class': RISK_COLUMN_NAME}, inplace=True)
    
    print(f"Successfully loaded {len(df_train)} instances of Training Data.")
    
    # 1. IDENTIFY THE BAD RISK CASES (Now correctly looking for the 'bad' label)
    # The 'bad' label is used based on your previous output.
    df_bad_risk = df_train[df_train[RISK_COLUMN_NAME].astype(str).str.lower() == 'bad']
        
    if df_bad_risk.empty:
        print("\nFATAL ERROR: Still unable to isolate 'bad' risk rows. Check your data labels.")
        exit()
        
    print("\n" + "="*70)
    print("CORRECTED ANALYSIS: FINDING PATTERNS IN BAD RISK LOANS (for Rule Tuning in Cell 1)")
    print("="*70)
    
    # 2. Numeric Analysis (Credit Amount, Duration)
    print("\n1. Numeric Analysis (to tune HIGH_AMOUNT_THRESHOLD & LONG_DURATION_THRESHOLD):")
    
    # We use the 75th percentile to identify the top 25% of the riskiest loans.
    high_amount_q75 = df_bad_risk['Credit amount'].quantile(0.75)
    long_duration_q75 = df_bad_risk['Duration'].quantile(0.75)
    
    print(f"   - **Threshold Suggestion (Credit Amount):** \t€{high_amount_q75:,.2f}")
    print(f"     (75% of 'bad' loans were greater than this amount)")
    print(f"   - **Threshold Suggestion (Duration):** \t{long_duration_q75:.2f} months")
    print(f"     (75% of 'bad' loans were longer than this duration)")

    # 3. Categorical Analysis (Purpose)
    print("\n2. Categorical Analysis (The top 5 purposes leading to Bad Risk):")
    purpose_counts = df_bad_risk['Purpose'].value_counts(normalize=True) * 100
    
    print("   - Top 5 Purposes in BAD Risk Loans (Proportion of all 'bad' loans):")
    print(purpose_counts.head(5).to_string(header=False, float_format="%.1f%%"))
    
    print("\n--- Use these numbers to refine your rules in Cell 1! ---")

except FileNotFoundError:
    print(f"\nFATAL ERROR: Training Data file not found at {FILE_PATH_TRAINING}. Please verify the path.")
except Exception as e:
    print(f"\nERROR during analysis: {e}")

Successfully loaded 1120 instances of Training Data.

CORRECTED ANALYSIS: FINDING PATTERNS IN BAD RISK LOANS (for Rule Tuning in Cell 1)

1. Numeric Analysis (to tune HIGH_AMOUNT_THRESHOLD & LONG_DURATION_THRESHOLD):
   - **Threshold Suggestion (Credit Amount):** 	€5,026.25
     (75% of 'bad' loans were greater than this amount)
   - **Threshold Suggestion (Duration):** 	36.00 months
     (75% of 'bad' loans were longer than this duration)

2. Categorical Analysis (The top 5 purposes leading to Bad Risk):
   - Top 5 Purposes in BAD Risk Loans (Proportion of all 'bad' loans):
car                   35.7%
radio/TV              20.2%
furniture/equipment   19.5%
business              10.4%
education              8.2%

--- Use these numbers to refine your rules in Cell 1! ---


In [17]:
# --- NEW CELL: TARGET COLUMN INSPECTION ---

# **CRITICAL**: Update this path to the ABSOLUTE PATH of your TRAINING DATA file.
FILE_PATH_TRAINING = r'C:\Users\Joel_\Downloads\AI-assessment-1\training_data.csv' 
RISK_COLUMN_NAME = 'Risk' 

try:
    df_train = pd.read_csv(FILE_PATH_TRAINING)
    
    # --- Attempt to find the outcome column ---
    if 'Class' in df_train.columns:
        df_train.rename(columns={'Class': RISK_COLUMN_NAME}, inplace=True)
    elif RISK_COLUMN_NAME not in df_train.columns:
        # Fallback: Check all columns for any binary/categorical data types
        print("WARNING: Could not find 'Risk' or 'Class' column. Here are the last few column names:")
        print(df_train.columns[-3:].tolist())
        # The user must manually identify the outcome column if this happens
        
    print(f"Successfully loaded {len(df_train)} instances of Training Data.")

    print("\n" + "="*50)
    print(f"OUTCOME COLUMN: '{RISK_COLUMN_NAME}' VALUE COUNTS")
    print("="*50)
    
    # This will show the actual labels and how many times they appear
    if RISK_COLUMN_NAME in df_train.columns:
        print(df_train[RISK_COLUMN_NAME].value_counts())
    else:
        print(f"ERROR: Column '{RISK_COLUMN_NAME}' is still missing after renaming attempts.")

except FileNotFoundError:
    print(f"\nFATAL ERROR: Training Data file not found at {FILE_PATH_TRAINING}. Please verify the path.")
except Exception as e:
    print(f"\nERROR during inspection: {e}")

Successfully loaded 1120 instances of Training Data.

OUTCOME COLUMN: 'Risk' VALUE COUNTS
Risk
bad     560
good    560
Name: count, dtype: int64
